In [1]:
import nemo.collections.asr as nemo_asr
from ruamel.yaml import YAML
from omegaconf import DictConfig
import numpy as np
from nemo.collections.asr.models.ctc_models import EncDecCTCModel
from nemo.collections.asr.metrics import wer
import tempfile
import os
import json
import torch

################################################################################
### WARNING, path does not exist: KALDI_ROOT=/mnt/matylda5/iveselyk/Tools/kaldi-trunk
###          (please add 'export KALDI_ROOT=<your_path>' in your $HOME/.profile)
###          (or run as: KALDI_ROOT=<your_path> python <your_script>.py)
################################################################################

[NeMo W 2022-06-24 08:03:25 experimental:27] Module <class 'nemo.collections.asr.data.audio_to_text_dali.AudioToCharDALIDataset'> is experimental, not ready for production and is not fully supported. Use at your own risk.


# Load fined-tuned model checkpoint

In [ ]:
model_to_load = "/TURKISH_FINETUNING_Quartznet15x5_models/epoch=XX-step=XXXXXX.ckpt"
stt_config_path = "/configs/quartznet15x5.yaml"
yaml = YAML(typ='safe')
with open(stt_config_path, encoding="utf-8") as f:
    params = yaml.load(f)

params['model']['train_ds']['manifest_filepath'] = "data/train_manifest.jsonl"
params['model']['validation_ds']['manifest_filepath'] = "data/val_manifest.jsonl"
first_asr_model = nemo_asr.models.EncDecCTCModel(cfg=DictConfig(params['model']))

first_asr_model = first_asr_model.load_from_checkpoint(model_to_load)
first_asr_model.eval()

# Create onnx model

In [ ]:
filename = "turkish_fine-tuned_model.onnx"
first_asr_model.export(output=filename)

# Load test file

In [44]:
import json
import os
test_files = []
test_texts = []
manifest_path = 'data/test_manifest.jsonl'
with open(manifest_path, 'r', encoding="utf-8") as f:  
    for line in f:
        original_file_json = json.loads(line)
        test_files.append(original_file_json["audio_filepath"]) 
        test_texts.append(original_file_json["text"])

In [47]:
import IPython.display as ipd

from nemo.collections.asr.parts import perturb, segment
# NeMo has its own support class for loading wav files
def load_audio(filepath) -> segment.AudioSegment:
    sample_segment = segment.AudioSegment.from_file(filepath, target_sr=16000)
    return sample_segment

In [48]:
filepath = "test_files/test.wav"
sample_segment = load_audio(filepath)
ipd.Audio(sample_segment.samples, rate=16000)

# Inference

In [ ]:
test_inference = first_asr_model.transcribe(test_files)
# print("Inference test:", test_inference)


for path, ref, pred in zip(test_files, test_texts, test_inference):
    print("path: {} \n reference: {} \n prediction:{}".format(path, ref, pred))

# Evaluate WER (Word Error Rate)

In [31]:
# Bigger batch-size = bigger throughput
params['model']['validation_ds']['batch_size'] = 16
params['model']['validation_ds']['manifest_filepath'] = "/data/val_manifest.jsonl"

# Setup the test data loader and make sure the model is on GPU
first_asr_model.setup_test_data(test_data_config=params['model']['validation_ds'])
first_asr_model.cpu()

# We will be computing Word Error Rate (WER) metric between our hypothesis and predictions.
# WER is computed as numerator/denominator.
# We'll gather all the test batches' numerators and denominators.
wer_nums = []
wer_denoms = []

# Loop over all test batches.
# Iterating over the model's `test_dataloader` will give us:
# (audio_signal, audio_signal_length, transcript_tokens, transcript_length)
# See the AudioToCharDataset for more details.
for test_batch in first_asr_model.test_dataloader():
        test_batch = [x.cpu() for x in test_batch]
        targets = test_batch[2]
        targets_lengths = test_batch[3]        
        log_probs, encoded_len, greedy_predictions = first_asr_model(
            input_signal=test_batch[0], input_signal_length=test_batch[1]
        )
        # Notice the model has a helper object to compute WER
        first_asr_model._wer.update(greedy_predictions, targets, targets_lengths)
        _, wer_num, wer_denom = first_asr_model._wer.compute()
        first_asr_model._wer.reset()
        wer_nums.append(wer_num.detach().cpu().numpy())
        wer_denoms.append(wer_denom.detach().cpu().numpy())

        # Release tensors from GPU memory
        del test_batch, log_probs, targets, targets_lengths, encoded_len, greedy_predictions

# We need to sum all numerators and denominators first. Then divide.
print(f"WER = {sum(wer_nums)/sum(wer_denoms)}")

[NeMo I 2022-04-26 11:01:11 collections:173] Dataset loaded with 201 files totalling 0.25 hours
[NeMo I 2022-04-26 11:01:11 collections:174] 0 files were filtered totalling 0.00 hours
WER = 0.05811277330264672
